In [15]:
import urllib3
import json
import time
import networkx as nx
import matplotlib.pyplot as plt
import os
import threading


CONTROLLER_IP = 'http://10.128.237.106:8080'

def get_all_switches():

    url = CONTROLLER_IP + '/stats/switches'
    http = urllib3.PoolManager()
    res = http.request('GET', url)
    res = res.data.decode('unicode_escape')
    res = json.loads(res)

    return res

def get_flow_switche_dpid():
# get flows stats of the switch filtered by the fields
    url = CONTROLLER_IP + '/stats/switches'
    http = urllib3.PoolManager()
    res = http.request('GET', url)
    res = res.data.decode('unicode_escape')
    res = json.loads(res)

    return res


def get_ground_port(num:int):
    url = CONTROLLER_IP + '/stats/port/{}'.format(num)
    http = urllib3.PoolManager()
    try:
        res = http.request('GET', url)
        if res.status == 200:
            res = res.data.decode('unicode_escape')
            res = json.loads(res)
        else:
            res = None
    except urllib3.exceptions.ResponseError as e:
            print("get error", e)

    return res


#  80,100,81,1,1
post_data_ip = {
    "dpid": 3,
    "table_id": 0,
    "idle_timeout": 0,
    "hard_timeout": 0,
    "priority": 1,
    "flags": 1,
    "match":{
        "ipv4_dst": "10.0.0.2",
        "eth_type": 2048
    },
    "actions":[
        {
            "type":"OUTPUT",
            "port": 1
        }
    ]
 }

post_data_arp = {
    "dpid": 1,
    "table_id": 0,
    "idle_timeout": 0,
    "hard_timeout": 0,
    "priority": 3,
    "flags": 1,
    "match":{
         "eth_type": 2054,
         "arp_tpa": "10.0.0.2"
    },
    "actions":[
        {
            "type":"OUTPUT",
            "port": 1
        }
    ]
 }


add_flowentry_url = CONTROLLER_IP + "/stats/flowentry/add"

fstates_dir = 'gen_data/Lion_isls_plus_grid_twostation_algorithm_free_one_only_over_isls/dynamic_state_100000ms_for_600s'
fstates_list = sorted(os.listdir(fstates_dir))

test_file = ['gen_data/Lion_isls_plus_grid_twostation_algorithm_free_one_only_over_isls/dynamic_state_100000ms_for_600s/fstate_0.txt']

# 60,101,61,1,1
# 61,100,51,0,3
# 2,100,92,3,3
# 2,101,92,3,3
priority_add_flow_update_router = 1

http = urllib3.PoolManager()

# 0,100,1,0,0
# 0,101,1,0,0
# 1,100,91,3,3
# 1,101,2,1,0
# for fstate in fstates_list[:len(fstates_list)//2]:
for fstate in test_file:
    priority_add_flow_update_router +=  1

    print('file= ', fstate)
    # with open(os.path.join(fstates_dir, fstate) , 'r') as f:
    with open(os.path.join(fstate) , 'r') as f:
        lines = f.readlines()

        for line in lines:
            print('add forward tabele : {}'.format(line))
            line_list = [int(i) for i in line.split(',')]
            src_dpid = line_list[0] + 1
            des_dpid = line_list[1] + 1
            next_hop_dpid = line_list[2] + 1
            output_port = line_list[3] + 1
            next_hop_input_port = line_list[4] + 1

            post_data_ip['dpid'] = src_dpid
            post_data_ip['priority'] = priority_add_flow_update_router
            post_data_ip["actions"][0]['port'] = output_port
            post_data_ip['match']['ipv4_dst'] = '10.0.{}.{}'.format(des_dpid // 255, des_dpid % 255)

            print(post_data_ip)
            encoded_data = json.dumps(post_data_ip).encode("utf-8")

            try :
                res = http.request(
                "POST",
                add_flowentry_url,
                body = encoded_data,
                headers = {
                    'x-env-code':'mafutian',
                    'content-type':'application/json;charset=UTF-8'})
                if res.status == 200:
                    response = res.data
                    if len(response) > 0:
                         response = json.loads(response)
                else:
                    response = None
            except urllib3.exceptions.ResponseError as e:
                print("post error", e)

            # time.sleep(1)

            post_data_arp['dpid'] = src_dpid
            post_data_arp['priority'] = priority_add_flow_update_router
            post_data_arp["actions"][0]['port'] = output_port
            post_data_arp['match']['arp_tpa'] = '10.0.{}.{}'.format(des_dpid // 255, des_dpid % 255)

            print(post_data_arp)
            encoded_data = json.dumps(post_data_arp).encode("utf-8")

            try :
                res = http.request(
                "POST",
                add_flowentry_url,
                body = encoded_data,
                headers = {
                    'x-env-code':'mafutian',
                    'content-type':'application/json;charset=UTF-8'})
                if res.status == 200:
                    response = res.data
                    if len(response) > 0:
                         response = json.loads(response)
                else:
                    response = None
            except urllib3.exceptions.ResponseError as e:
                print("post error", e)

            # time.sleep(1)







file=  gen_data/Lion_isls_plus_grid_twostation_algorithm_free_one_only_over_isls/dynamic_state_100000ms_for_600s/fstate_0.txt
add forward tabele : 0,100,1,0,0

{'dpid': 1, 'table_id': 0, 'idle_timeout': 0, 'hard_timeout': 0, 'priority': 2, 'flags': 1, 'match': {'ipv4_dst': '10.0.0.101', 'eth_type': 2048}, 'actions': [{'type': 'OUTPUT', 'port': 1}]}
{'dpid': 1, 'table_id': 0, 'idle_timeout': 0, 'hard_timeout': 0, 'priority': 2, 'flags': 1, 'match': {'eth_type': 2054, 'arp_tpa': '10.0.0.101'}, 'actions': [{'type': 'OUTPUT', 'port': 1}]}
add forward tabele : 0,101,1,0,0

{'dpid': 1, 'table_id': 0, 'idle_timeout': 0, 'hard_timeout': 0, 'priority': 2, 'flags': 1, 'match': {'ipv4_dst': '10.0.0.102', 'eth_type': 2048}, 'actions': [{'type': 'OUTPUT', 'port': 1}]}
{'dpid': 1, 'table_id': 0, 'idle_timeout': 0, 'hard_timeout': 0, 'priority': 2, 'flags': 1, 'match': {'eth_type': 2054, 'arp_tpa': '10.0.0.102'}, 'actions': [{'type': 'OUTPUT', 'port': 1}]}
add forward tabele : 1,100,91,3,3

{'dpid': 

#### add ground to host links

In [10]:
import time
import urllib3
import json

CONTROLLER_IP = 'http://10.128.237.106:8080'
add_flowentry_url = CONTROLLER_IP + "/stats/flowentry/add"


post_data_ip = {
    "dpid": 3,
    "table_id": 0,
    "idle_timeout": 0,
    "hard_timeout": 0,
    "priority": 1,
    "flags": 1,
    "match":{
        "ipv4_dst": "10.0.0.2",
        "eth_type": 2048
    },
    "actions":[
        {
            "type":"OUTPUT",
            "port": 1
        }
    ]
 }

post_data_arp = {
    "dpid": 1,
    "table_id": 0,
    "idle_timeout": 0,
    "hard_timeout": 0,
    "priority": 3,
    "flags": 1,
    "match":{
         "eth_type": 2054,
         "arp_tpa": "10.0.0.2"
    },
    "actions":[
        {
            "type":"OUTPUT",
            "port": 1
        }
    ]
 }


def get_ground_port(num:int):
    url = CONTROLLER_IP + '/stats/port/{}'.format(num)
    http = urllib3.PoolManager()
    try:
        res = http.request('GET', url)
        if res.status == 200:
            res = res.data.decode('unicode_escape')
            res = json.loads(res)
        else:
            res = None
    except urllib3.exceptions.ResponseError as e:
            print("get error", e)

    return res

http = urllib3.PoolManager()
####  add ground to host link
# ground_list = [101, 102]
ground_list = [226]
for gs_node_num in ground_list:

    res = get_ground_port(gs_node_num)  # num -> ground number
    gs_port_list = res['{}'.format(gs_node_num)]
    gs_port_list = [port_info['port_no'] for port_info in gs_port_list if type(port_info['port_no']) == int]

    gs_port_list  = sorted(gs_port_list)
    print(gs_port_list)
    # print(sorted(port_list))

    host_eth_num = gs_port_list[-1]  #最大的端口号，因为host是最后建立的连接


    post_data_ip['dpid'] = gs_node_num
    post_data_ip['priority'] = 2
    post_data_ip["actions"][0]['port'] = host_eth_num
    post_data_ip['match']['ipv4_dst'] = '10.0.{}.{}'.format(gs_node_num//255, gs_node_num%255)

    encoded_data = json.dumps(post_data_ip).encode("utf-8")

#
    try :
        res = http.request(
        "POST",
        add_flowentry_url,
        body = encoded_data,
        headers = {
            'x-env-code':'mafutian',
            'content-type':'application/json;charset=UTF-8'})
        if res.status == 200:
            response = res.data
            if len(response) > 0:
                 response = json.loads(response)
        else:
            response = None
    except urllib3.exceptions as e:
        print("post error", e)

    # time.sleep(1)


    post_data_arp['dpid'] = gs_node_num
    post_data_arp['priority'] = 2
    post_data_arp["actions"][0]['port'] = host_eth_num
    post_data_arp['match']['arp_tpa'] = '10.0.{}.{}'.format(gs_node_num//255, gs_node_num%255)

    encoded_data = json.dumps(post_data_arp).encode("utf-8")


    try :
        res = http.request(
        "POST",
        add_flowentry_url,
        body = encoded_data,
        headers = {
            'x-env-code':'mafutian',
            'content-type':'application/json;charset=UTF-8'})
        if res.status == 200:
            response = res.data
            if len(response) > 0:
                 response = json.loads(response)
        else:
            response = None
    except urllib3.exceptions as e:
        print("post error", e)




[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]
81


#### topo topo 1–>2–>3–>4–>1

In [5]:
import time
import json
import urllib3

CONTROLLER_IP = 'http://10.128.237.106:8080'
add_flowentry_url = CONTROLLER_IP + "/stats/flowentry/add"

post_data_ip = {
    "dpid": 3,
    "table_id": 0,
    "idle_timeout": 0,
    "hard_timeout": 0,
    "priority": 1,
    "flags": 1,
    "match":{
        "ipv4_dst": "10.0.0.2",
        "eth_type": 2048
    },
    "actions":[
        {
            "type":"OUTPUT",
            "port": 1
        }
    ]
 }

post_data_arp = {
    "dpid": 1,
    "table_id": 0,
    "idle_timeout": 0,
    "hard_timeout": 0,
    "priority": 3,
    "flags": 1,
    "match":{
         "eth_type": 2054,
         "arp_tpa": "10.0.0.2"
    },
    "actions":[
        {
            "type":"OUTPUT",
            "port": 1
        }
    ]
 }

dpid = [1,2,3,4]

dst1_output = [3,1,1,1]
dst2_output = [1, 2, 2,3]

http = urllib3.PoolManager()

for i in dpid:

    post_data_ip['dpid'] = i
    post_data_ip['priority'] = 44
    post_data_ip["actions"][0]['port'] = dst1_output[i-1]
    post_data_ip['match']['ipv4_dst'] = '10.0.0.1'

    print(post_data_ip)

    encoded_data = json.dumps(post_data_ip).encode("utf-8")


    try :
        res = http.request(
        "POST",
        add_flowentry_url,
        body = encoded_data,
        headers = {
            'x-env-code':'mafutian',
            'content-type':'application/json;charset=UTF-8'})
        if res.status == 200:
            response = res.data
            if len(response) > 0:
                 response = json.loads(response)
        else:
            response = None
    except urllib3.exceptions as e:
        print("post error", e)

    time.sleep(1)

    post_data_ip['dpid'] = i
    post_data_ip['priority'] = 44
    post_data_ip["actions"][0]['port'] = dst2_output[i-1]
    post_data_ip['match']['ipv4_dst'] = '10.0.0.2'

    print(post_data_ip)
    encoded_data = json.dumps(post_data_ip).encode("utf-8")

    try :
        res = http.request(
        "POST",
        add_flowentry_url,
        body = encoded_data,
        headers = {
            'x-env-code':'mafutian',
            'content-type':'application/json;charset=UTF-8'})
        if res.status == 200:
            response = res.data
            if len(response) > 0:
                 response = json.loads(response)
        else:
            response = None
    except urllib3.exceptions.ResponseError as e:
        print("post error", e)
    time.sleep(1)


for i in dpid:
    post_data_arp['dpid'] = i
    post_data_arp['priority'] = 44
    post_data_arp["actions"][0]['port'] = dst1_output[i-1]
    post_data_arp['match']['arp_tpa'] = '10.0.0.1'

    print(post_data_arp)
    encoded_data = json.dumps(post_data_arp).encode("utf-8")


    try :
        res = http.request(
        "POST",
        add_flowentry_url,
        body = encoded_data,
        headers = {
            'x-env-code':'mafutian',
            'content-type':'application/json;charset=UTF-8'})
        if res.status == 200:
            response = res.data
            if len(response) > 0:
                 response = json.loads(response)
        else:
            response = None
    except urllib3.exceptions as e:
        print("post error", e)

    # time.sleep(1)

    post_data_arp['dpid'] = i
    post_data_arp['priority'] = 44
    post_data_arp["actions"][0]['port'] = dst2_output[i-1]
    post_data_arp['match']['arp_tpa'] = '10.0.0.2'

    print(post_data_arp)
    encoded_data = json.dumps(post_data_arp).encode("utf-8")

    try :
        res = http.request(
        "POST",
        add_flowentry_url,
        body = encoded_data,
        headers = {
            'x-env-code':'mafutian',
            'content-type':'application/json;charset=UTF-8'})
        if res.status == 200:
            response = res.data
            if len(response) > 0:
                 response = json.loads(response)
        else:
            response = None
    except urllib3.exceptions.ResponseError as e:
        print("post error", e)

    # time.sleep(1)

{'dpid': 1, 'table_id': 0, 'idle_timeout': 0, 'hard_timeout': 0, 'priority': 44, 'flags': 1, 'match': {'ipv4_dst': '10.0.0.1', 'eth_type': 2048}, 'actions': [{'type': 'OUTPUT', 'port': 3}]}
{'dpid': 1, 'table_id': 0, 'idle_timeout': 0, 'hard_timeout': 0, 'priority': 44, 'flags': 1, 'match': {'ipv4_dst': '10.0.0.2', 'eth_type': 2048}, 'actions': [{'type': 'OUTPUT', 'port': 1}]}
{'dpid': 2, 'table_id': 0, 'idle_timeout': 0, 'hard_timeout': 0, 'priority': 44, 'flags': 1, 'match': {'ipv4_dst': '10.0.0.1', 'eth_type': 2048}, 'actions': [{'type': 'OUTPUT', 'port': 1}]}
{'dpid': 2, 'table_id': 0, 'idle_timeout': 0, 'hard_timeout': 0, 'priority': 44, 'flags': 1, 'match': {'ipv4_dst': '10.0.0.2', 'eth_type': 2048}, 'actions': [{'type': 'OUTPUT', 'port': 2}]}
{'dpid': 3, 'table_id': 0, 'idle_timeout': 0, 'hard_timeout': 0, 'priority': 44, 'flags': 1, 'match': {'ipv4_dst': '10.0.0.1', 'eth_type': 2048}, 'actions': [{'type': 'OUTPUT', 'port': 1}]}
{'dpid': 3, 'table_id': 0, 'idle_timeout': 0, 'har